In [55]:
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import easygems.healpix as egh
import cartopy.crs as ccrs
import healpy as hp
import numpy as np
import pandas as pd

In [8]:
def get_nn_lon_lat_index(nside, lons, lats):
    """
    nside: integer, power of 2. The return of hp.get_nside()
    lons: uniques values of longitudes
    lats: uniques values of latitudes
    returns: array with the HEALPix cells that are closest to the lon/lat grid
    """
    lons2, lats2 = np.meshgrid(lons, lats)
    return xr.DataArray(
        hp.ang2pix(nside, lons2, lats2, nest = True, lonlat = True),
        coords=[('latitude', lats), ('longitude', lons)],
    )

In [96]:
zarr_file = "/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT1H.z10.zarr"
ds = xr.open_zarr(zarr_file)

In [97]:
var_um = ds['pr']

In [98]:
# Find the HEALPix pixels that are closest to the ERA5 grid
res = 0.04
# longitudes and latitudes for the ERA5 grid
lon = np.arange(130.5,131.5+res,res)
lat = np.arange(-13, -12+res,res)

# nside for um simulation, it should be equal to 2**zoom
this_nside = hp.get_nside(var_um)

cells = get_nn_lon_lat_index(this_nside, lon, lat) 

var_um_regrided = var_um.isel(cell = cells)#.rename({'lon': 'longitude', 'lat': 'latitude'}) # regriding


In [99]:
projection = ccrs.PlateCarree(central_longitude = 0.0)
fig, ax = plt.subplots(subplot_kw = {"projection": projection})
var_um_regrided.isel(time = 300).plot(cmap = 'viridis', transform = ccrs.PlateCarree())
ax.coastlines()

In [100]:

# Create full date range from 2020-03-01 to 2021-02-28
full_range = pd.date_range(start="2020-03-01", end="2021-02-28", freq="h")

# Define exclusion period
exclude_start = pd.Timestamp("2020-05-01 00:00:00")
exclude_end = pd.Timestamp("2020-09-30 00:00:00")

# Filter out the excluded range
filtered_range = full_range[(full_range < exclude_start) | (full_range > exclude_end)]

# Convert to list of strings in yyyymmdd format
date_str_list = filtered_range.strftime("%Y%m%d %H:%M:%S").tolist()

In [101]:
%%time
var_um_regrided.sel(time=date_str_list).to_netcdf("/scratch/nf33/hk25_LSP/UK/Darwin/pr_hourly.nc")

CPU times: user 33.1 s, sys: 9 s, total: 42.1 s
Wall time: 22 s
